In [29]:
import pandas as pd
import re

#Import data

In [30]:
df = pd.read_csv('mpg.csv')

In [31]:
def mpg_kml(x):
    return round(x * 0.425144,2)


In [32]:
df['cty_kml'] = df['cty'].apply(mpg_kml)

In [33]:
df['trans'] = df['trans'].apply(lambda x:  'auto' if re.search(r'\b[Aa]',x) else 'manual' )

In [34]:
df['drv'].unique()

array(['f', '4', 'r'], dtype=object)

In [35]:
df.drv = df.drv.map({'f': 'FRONT', 
                    '4': '4WD', 
                    'r':'READ'})

In [36]:
df

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,cty_kml
0,audi,a4,1.8,1999,4,auto,FRONT,18,29,p,compact,7.65
1,audi,a4,1.8,1999,4,manual,FRONT,21,29,p,compact,8.93
2,audi,a4,2.0,2008,4,manual,FRONT,20,31,p,compact,8.50
3,audi,a4,2.0,2008,4,auto,FRONT,21,30,p,compact,8.93
4,audi,a4,2.8,1999,6,auto,FRONT,16,26,p,compact,6.80
...,...,...,...,...,...,...,...,...,...,...,...,...
229,volkswagen,passat,2.0,2008,4,auto,FRONT,19,28,p,midsize,8.08
230,volkswagen,passat,2.0,2008,4,manual,FRONT,21,29,p,midsize,8.93
231,volkswagen,passat,2.8,1999,6,auto,FRONT,16,26,p,midsize,6.80
232,volkswagen,passat,2.8,1999,6,manual,FRONT,18,26,p,midsize,7.65
